# Домашнее задание

### Д/з из четырех пунктов:
* Улучшение `fit_generator`
* Сравнение двух ReLU (разные активации)
* Испорченный батч-норм 
* "Сырые" данные. 

### Что нужно сделать
* Следовать инструкциям в каждом из пунктов.
* Результатами вашей работы будет ноутбук с доработанным кодом + архив с директорией с логами `tensorboard` `logs/`, в который вы запишите результаты экспериментов. Подробности в инструкциях ниже.
* Можно и нужно пользоваться кодом из файла `utils`, **но** весь код модифицируйте, пожалуйста, в ноутбуках! Так мне будет проще проверять.

**Загрузка tensorboard в ноутбук**

Можете попробовать использовать его так на свой страх и риск :)

In [1]:
#%load_ext tensorboard
#%tensorboard --logdir logs

**Импорты**

In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from typing import Callable

### Импорт слоев для д/з

In [4]:
from utils import BatchNormFlawed, Dense, DenseSmart, Sequential, MNISTSequence

### Загрузка данных

> Здесь ничего менять не нужно

In [5]:
(X_tr, y_tr), (X_test, y_test) = keras.datasets.mnist.load_data()

In [6]:
train_seq = MNISTSequence(X_tr, y_tr, 128)
test_seq = MNISTSequence(X_test, y_test, 128)

**Очистка данных**

In [7]:
import errno, os, stat, shutil

In [8]:
def handle_remove_readonly(func, path, exc):
    excvalue = exc[1]
    if func in (os.rmdir, os.remove) and excvalue.errno == errno.EACCES:
        os.chmod(path, stat.S_IRWXU| stat.S_IRWXG| stat.S_IRWXO) # 0777
        func(path)
    else:
        raise

In [11]:
! rm -r logs

In [12]:
mkdir logs

In [13]:
keras.backend.clear_session()

## 1. Улучшение fit_generator

Улучшите метод `fit_generator` так, чтобы он:
* Записывал значения градиентов для всех переменных при помощи `tf.summary.histogram` 
* Записывал значения ошибки и метрики на валидации с помощью `tf.summary.scalar`

Затем сделайте monkey patch класса sequential обновленным методом (следующая ячейка за методом `fit_generator`).

In [14]:
def fit_generator(self, train_seq, eval_seq, epoch, loss, optimizer, writer=None, log_freq=50):
    history = dict(train=list(), val=list())

    train_loss_results = list()
    val_loss_results = list()

    train_accuracy_results = list()
    val_accuracy_results = list()

    step = 0
    for e in range(epoch):
        p = tf.keras.metrics.Mean()
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_loss_avg_val = tf.keras.metrics.Mean()

        epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        epoch_accuracy_val = tf.keras.metrics.SparseCategoricalAccuracy()

        for x, y in train_seq:
            with tf.GradientTape() as tape:
                """
                Обратите внимание! Если записывать гистограмму каждый шаг,
                обучение будет идти очень медленно. Поэтому записываем данные 
                каждый i-й шаг.
                """
                if step % log_freq == 0:
                    prediction = self._forward(x, writer, step)
                else:
                    prediction = self._forward(x)
                loss_value = loss(y, prediction)
                     
            ###############################################################
            #                                                             #
            # Добавьте запись градиентов в гистограммы                    #
            #                                                             #
            ###############################################################
        
            gradients = tape.gradient(loss_value, self._trainable_variables)
            
            if step % log_freq == 0:
                """
                Пример того, как можно дать всем градиентам уникальные имена. 
                Обратите внимание! Создание grad_names лучше вынести из цикла,
                чтобы не пересоздавать список на каждом шаге! 
                """
                grad_names = list()
                for layer in self._layers:
                    for var_num, var in enumerate(layer.get_trainable()):
                        grad_names.append(f"grad_{layer.name}_{var_num}")
                # сохраняем значения градиентов  
                if writer is not None:
                    with writer.as_default():
                        for name, grad in zip(grad_names, gradients):                    
                            tf.summary.histogram(name, grad, step=step)
                
            
            optimizer.apply_gradients(zip(gradients, self._trainable_variables))
            epoch_accuracy.update_state(y, prediction)
            epoch_loss_avg.update_state(loss_value)

            if step % log_freq == 0 and writer is not None:
                with writer.as_default():
                    # сохраняем значения метрики и функции потерь
                    tf.summary.scalar('train_accuracy', epoch_accuracy.result().numpy(), step=step)
                    tf.summary.scalar('train_loss', epoch_loss_avg.result().numpy(), step=step)

            step += 1

        train_accuracy_results.append(epoch_accuracy.result().numpy())
        train_loss_results.append(epoch_loss_avg.result().numpy())

        for x, y in eval_seq:
            prediction = self._forward(x)
            loss_value = loss(y, prediction)
            epoch_loss_avg_val.update_state(loss_value)
            epoch_accuracy_val.update_state(y, prediction)
     
            ###############################################################
            #                                                             #
            # Добавьте сохранение метрики и функции ошибки на валидации   #
            #                                                             #
            ###############################################################
            if step % 50 == 0 and writer is not None:
                with writer.as_default():
                    tf.summary.scalar('val_accuracy', epoch_accuracy_val.result().numpy(), )
                    tf.summary.scalar('val_loss', epoch_loss_avg_val.result().numpy(), step=step)
            
        val_accuracy_results.append(epoch_accuracy_val.result().numpy())
        val_loss_results.append(epoch_loss_avg_val.result().numpy())

        # print(f"Epoch train loss: {epoch_train_loss[-1]:.2f},\nEpoch val loss: {epoch_val_loss[-1]:.2f}\n{'-'*20}")
        print("Epoch {}: Train loss: {:.3f} Train Accuracy: {:.3f}".format(e + 1,
                                                                           train_loss_results[-1],
                                                                           train_accuracy_results[-1]))
        print("Epoch {}: Val loss: {:.3f} Val Accuracy: {:.3f}".format(e + 1,
                                                                       val_loss_results[-1],
                                                                       val_accuracy_results[-1]))
        print('*' * 20)

    return None

In [15]:
# Monkey patch: обновляем метод
Sequential.fit_generator = fit_generator

## 2. Сравнение двух ReLU (разные активации)

Запустите два эксперимента ниже. Сравните результаты - значения метрик после каждого из них.

Запустите tensorboard, изучите распределения активаций, градиентов и т.д. для `relu` и `smart_dense_relu`. 

Подумайте, почему в одном случае сеть обучается плохо, а в другом - хорошо. Вставьте в ноутбук (или напишите список названий) тех графики из tensorboard, которые, по вашему мнению, это иллюстрируют, и напишите, почему.


Команда для запуска tensorboard в bash:

`$ tensorboard --logdir logs/`

**Ваш комментарий:**

##### В слоях Dense начальные значения весов больше, чем в слоях SmartDense.
<img src="weights_dispersion.png" title="" />

##### В результате, больше дисперсия выходных сигналов нейронов, больше дисперсия сигналов активации, больше дисперсия функции потерь.
<img src="z_dispersion.png" title="" />

##### На гистограммах градиентов видно, что для модели Dense наблюдаются велчины градиента обоих знаков,
##### а для модели SmartDense - только одного знака.
<img src="gradient_dispersion.png" title="" />

##### это ознапает, что градиентный спуск для модели Dense сопровождается переходами на противоположные "склоны" поверхности функции потерь, а для модели SmartDense - постепенным спуском по одному и тому же склону.
##### Т.е. за одинаковое количество итераций модель SmartDense становится ближе к локальному минимуму функции потерь.
##### В результате, обучение модели SmartDense происходит быстрее.
<img src="learning_curve.png" title="" />
---------

In [16]:
writer = tf.summary.create_file_writer("logs/relu")

model = Sequential(Dense(784, 100, tf.nn.relu, 'dense'), 
                   Dense(100, 100, tf.nn.relu, 'dense1'), 
                   Dense(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer, log_freq=50)

Epoch 1: Train loss: 10.111 Train Accuracy: 0.369
Epoch 1: Val loss: 8.275 Val Accuracy: 0.483
********************
Epoch 2: Train loss: 7.291 Train Accuracy: 0.544
Epoch 2: Val loss: 6.795 Val Accuracy: 0.575
********************
Epoch 3: Train loss: 6.459 Train Accuracy: 0.597
Epoch 3: Val loss: 6.329 Val Accuracy: 0.606
********************
Epoch 4: Train loss: 6.121 Train Accuracy: 0.618
Epoch 4: Val loss: 5.992 Val Accuracy: 0.626
********************
Epoch 5: Train loss: 5.955 Train Accuracy: 0.629
Epoch 5: Val loss: 5.909 Val Accuracy: 0.632
********************
Epoch 6: Train loss: 5.829 Train Accuracy: 0.637
Epoch 6: Val loss: 5.844 Val Accuracy: 0.636
********************
Epoch 7: Train loss: 5.714 Train Accuracy: 0.644
Epoch 7: Val loss: 5.271 Val Accuracy: 0.670
********************
Epoch 8: Train loss: 4.775 Train Accuracy: 0.702
Epoch 8: Val loss: 4.555 Val Accuracy: 0.716
********************
Epoch 9: Train loss: 4.448 Train Accuracy: 0.722
Epoch 9: Val loss: 4.354 Val A

In [17]:
writer = tf.summary.create_file_writer("logs/relu_smart_dense")

model = Sequential(DenseSmart(784, 100, tf.nn.relu, 'dense'), 
                   DenseSmart(100, 100, tf.nn.relu, 'dense1'), 
                   DenseSmart(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

Epoch 1: Train loss: 0.361 Train Accuracy: 0.896
Epoch 1: Val loss: 0.187 Val Accuracy: 0.942
********************
Epoch 2: Train loss: 0.138 Train Accuracy: 0.959
Epoch 2: Val loss: 0.132 Val Accuracy: 0.957
********************
Epoch 3: Train loss: 0.095 Train Accuracy: 0.972
Epoch 3: Val loss: 0.114 Val Accuracy: 0.965
********************
Epoch 4: Train loss: 0.071 Train Accuracy: 0.979
Epoch 4: Val loss: 0.107 Val Accuracy: 0.966
********************
Epoch 5: Train loss: 0.054 Train Accuracy: 0.985
Epoch 5: Val loss: 0.108 Val Accuracy: 0.969
********************
Epoch 6: Train loss: 0.044 Train Accuracy: 0.987
Epoch 6: Val loss: 0.109 Val Accuracy: 0.970
********************
Epoch 7: Train loss: 0.034 Train Accuracy: 0.989
Epoch 7: Val loss: 0.106 Val Accuracy: 0.971
********************
Epoch 8: Train loss: 0.027 Train Accuracy: 0.992
Epoch 8: Val loss: 0.110 Val Accuracy: 0.972
********************
Epoch 9: Train loss: 0.027 Train Accuracy: 0.991
Epoch 9: Val loss: 0.106 Val Ac

## 3.a Испорченный батч-норм 

Запустите два эксперимент ниже. 

Почему обучение не идет? В чем ошибка в слое `BatchNorm`? Изучите и исправьте код метода `__call__` (Шаблон находится ниже под блоком с экспериментом.).

Можно пользоваться tensorboard, если он нужен.

## ReLU + Batch Norm

In [18]:
writer = tf.summary.create_file_writer("logs/relu_bn")

model = Sequential(Dense(784, 100, tf.nn.relu, 'dense'), 
                   BatchNormFlawed('batch_norm'), 
                   Dense(100, 100, tf.nn.relu, 'dense1'), 
                   Dense(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

Epoch 1: Train loss: nan Train Accuracy: 0.098
Epoch 1: Val loss: nan Val Accuracy: 0.098
********************
Epoch 2: Train loss: nan Train Accuracy: 0.099
Epoch 2: Val loss: nan Val Accuracy: 0.098
********************
Epoch 3: Train loss: nan Train Accuracy: 0.099
Epoch 3: Val loss: nan Val Accuracy: 0.098
********************
Epoch 4: Train loss: nan Train Accuracy: 0.099
Epoch 4: Val loss: nan Val Accuracy: 0.098
********************
Epoch 5: Train loss: nan Train Accuracy: 0.099
Epoch 5: Val loss: nan Val Accuracy: 0.098
********************
Epoch 6: Train loss: nan Train Accuracy: 0.099
Epoch 6: Val loss: nan Val Accuracy: 0.098
********************
Epoch 7: Train loss: nan Train Accuracy: 0.099
Epoch 7: Val loss: nan Val Accuracy: 0.098
********************
Epoch 8: Train loss: nan Train Accuracy: 0.099
Epoch 8: Val loss: nan Val Accuracy: 0.098
********************
Epoch 9: Train loss: nan Train Accuracy: 0.099
Epoch 9: Val loss: nan Val Accuracy: 0.098
********************
E

**Класс, который нужно исправить**

In [22]:
class BatchNormFixed(BatchNormFlawed):
    def __call__(self, x, writer=None, step=None, eps=0.0001):
        """
        Исправьте блок кода ниже так, чтобы модель обучалась, не появлялись значения loss = NaN        """
        mu = tf.reduce_mean(x, axis=0)
        sigma = tf.math.reduce_std(x, axis=0)
        normed = (x - mu) / (sigma + eps)
        out = normed * self._gamma + self._beta
        """
        Конец блока, который нужно исправить
        """
        
        if writer is not None:
            with writer.as_default():
                tf.summary.histogram(self.name + '_beta', self._beta, step=step)
                tf.summary.histogram(self.name + '_gamma', self._gamma, step=step)
                tf.summary.histogram(self.name + '_normed', normed, step=step)
                tf.summary.histogram(self.name + '_out', out, step=step)
                tf.summary.histogram(self.name + '_sigma', sigma, step=step)
                tf.summary.histogram(self.name + '_mu', mu, step=step)
        return out

## 3.b Исправленный батч-норм 

Запустите эксперимент ниже. 

Обучается ли сеть? Идет ли процесс обучения лучше, чем в эксперименте с ReLU? 

Сравните обучение сетей c ReLU и слоем `Dense` (а не `DenseSmart`!) и ReLU с BatchNorm в tensorboard, как в задании 2.
Напишите ваши выводы.

_Обратите внимание, что слева в интерфейсе tensorboard есть меню, которое позволяет выключать визуализацию ненужных экспериментов._  

**Ваш комментарий:**  
##### В коде BatchNorm нужно исправить деление на 0.
##### При нормализации происходит деление на стандартное отклонение, однако,
##### если его значение равно 0, происходит ошибка вычисления.
##### Для исправления к величине $\sigma$ добавляется небольшая величина $\epsilon$.
##### Начинают правильно вычисляться активации, функция ошибки, градиенты.
##### Сеть начинает обучаться.
<img src="fixed_batch_norm.png" title="" />
---------

In [23]:
writer = tf.summary.create_file_writer("logs/relu_bn_fixed")

model = Sequential(Dense(784, 100, tf.nn.relu, 'dense'), 
                   BatchNormFixed('batch_norm'), 
                   Dense(100, 100, tf.nn.relu, 'dense1'), 
                   Dense(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

Epoch 1: Train loss: 9.815 Train Accuracy: 0.355
Epoch 1: Val loss: 4.107 Val Accuracy: 0.670
********************
Epoch 2: Train loss: 1.329 Train Accuracy: 0.756
Epoch 2: Val loss: 0.628 Val Accuracy: 0.809
********************
Epoch 3: Train loss: 0.594 Train Accuracy: 0.821
Epoch 3: Val loss: 0.511 Val Accuracy: 0.843
********************
Epoch 4: Train loss: 0.491 Train Accuracy: 0.853
Epoch 4: Val loss: 0.435 Val Accuracy: 0.867
********************
Epoch 5: Train loss: 0.419 Train Accuracy: 0.874
Epoch 5: Val loss: 0.380 Val Accuracy: 0.883
********************
Epoch 6: Train loss: 0.366 Train Accuracy: 0.890
Epoch 6: Val loss: 0.339 Val Accuracy: 0.897
********************
Epoch 7: Train loss: 0.324 Train Accuracy: 0.903
Epoch 7: Val loss: 0.307 Val Accuracy: 0.906
********************
Epoch 8: Train loss: 0.290 Train Accuracy: 0.914
Epoch 8: Val loss: 0.283 Val Accuracy: 0.914
********************
Epoch 9: Train loss: 0.260 Train Accuracy: 0.922
Epoch 9: Val loss: 0.262 Val Ac

## 4. "Сырые" данные. 

Что будет, если заставить сеть обучаться на сырых данных? 

Напишите такую функцию `preprocess`, которая не делает min-max scaling изображений и оставляет их в изначальном диапазоне. Не убирайте reshape! Конечно, она должна менять форму матрицы входных данных от `(n x 28 x 28)` к `(n x 784)`. 

Затем передайте функцию в MNISTSequence, создайте новую train- и test- последовательности запустите эксперимент, используя их как входные данные. 

Сравните результаты экспериментов c `DenseSmart` + ReLU и обработанными изображениями и `DenseSmart` + ReLU c необработанными изображениями. 

Обучается ли нейросеть? Если нет, то почему? Сделайте выводы, как в задании 2.

**Ваш комментарий:**
##### Обучение не происходит.
<img src="stuck_learning.png" title="" />

##### Выходные сигналы нейронов стали принимать преимущественно отрицательные значения.
<img src="negative_activation.png" title="" />

##### Как результат - нулевая производная функции активации ReLU, и, соответсвенно,
##### нулевые граденты.
<img src="zero_gradients.png" title="" />
---------

**Шаблон Preprocess**

In [25]:
def preprocess(X, y):
    return X.reshape((-1, 28*28)), y

**Создание генераторов**

In [26]:
train_seq_raw = MNISTSequence(X_tr, y_tr, 128, preprocess=preprocess)
test_seq_raw = MNISTSequence(X_test, y_test, 128, preprocess=preprocess)

**Эксперимент**

In [27]:
writer = tf.summary.create_file_writer("logs/raw")

model = Sequential(DenseSmart(784, 100, tf.nn.relu, 'dense'), 
                   DenseSmart(100, 100, tf.nn.relu, 'dense1'), 
                   DenseSmart(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq_raw, test_seq_raw, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer
                          )

Epoch 1: Train loss: 11.997 Train Accuracy: 0.256
Epoch 1: Val loss: 11.684 Val Accuracy: 0.275
********************
Epoch 2: Train loss: 11.805 Train Accuracy: 0.268
Epoch 2: Val loss: 11.632 Val Accuracy: 0.278
********************
Epoch 3: Train loss: 11.876 Train Accuracy: 0.263
Epoch 3: Val loss: 11.847 Val Accuracy: 0.265
********************
Epoch 4: Train loss: 11.615 Train Accuracy: 0.279
Epoch 4: Val loss: 11.566 Val Accuracy: 0.282
********************
Epoch 5: Train loss: 11.736 Train Accuracy: 0.272
Epoch 5: Val loss: 12.121 Val Accuracy: 0.248
********************
Epoch 6: Train loss: 11.772 Train Accuracy: 0.270
Epoch 6: Val loss: 11.623 Val Accuracy: 0.279
********************
Epoch 7: Train loss: 11.712 Train Accuracy: 0.273
Epoch 7: Val loss: 11.642 Val Accuracy: 0.278
********************
Epoch 8: Train loss: 11.774 Train Accuracy: 0.269
Epoch 8: Val loss: 11.858 Val Accuracy: 0.264
********************
Epoch 9: Train loss: 11.664 Train Accuracy: 0.276
Epoch 9: Val l